In [78]:
%load_ext autoreload
%autoreload 2

from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from pathlib import Path

MODEL_ID = "sshleifer/tiny-gpt2"  # tiny model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"

CACHE_DIR = Path("./store/tinystories")
STORE_DIR = Path("./store/tiny-gpt2")
DEVICE = 'cpu'
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 200  # keep small for a quick demo
MAX_LENGTH = 128

model = LanguageModel.from_huggingface(MODEL_ID)
model.get_model().to(DEVICE)

dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 96709.80 examples/s] 


In [79]:
import os
from amber.mechanistic.autoencoder.autoencoder import Autoencoder

sae_models = STORE_DIR / "sae_models"
last_run = os.listdir(sae_models)[-1]

sae_layer = Autoencoder.load_model(sae_models / last_run / 'final.pt')[0]
sae_layer.metadata

2025-10-13 14:38:59,225 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from store/tiny-gpt2/sae_models/tinystories_20250930_203105/final.pt
n_latents=2, n_inputs=2, activation=TopK_4, tied=False


{'run_name': 'tinystories_20250930_203105',
 'model': 'GPT2LMHeadModel',
 'layer_signature': 'gpt2lmheadmodel_transformer_h_1_ln_2',
 'dataset': {'cache_dir': 'store/tinystories', 'length': 200},
 'options': {'dtype': None,
  'max_length': 128,
  'save_inputs': True,
  'batch_size': 8}}

In [80]:
model_name = sae_layer.metadata['model']
layer_signature = sae_layer.metadata['layer_signature']

model = LanguageModel.from_huggingface(
    "sshleifer/tiny-gpt2",
)

In [81]:
model.layers.register_new_layer('sae', sae_layer, layer_signature)

In [82]:
sae_layer.concepts.enable_text_tracking()

In [83]:
dataset = TextSnippetDataset.from_huggingface(
    "roneneldan/TinyStories",
    split="train",
    cache_dir="./cache/tinystories",
    text_field="text",
    limit=100
)

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 59451.51 examples/s]


In [84]:
texts = dataset[0:5]

In [85]:
res = model.forwards(texts)

In [86]:
print(sae_layer.concepts.top_texts_tracker.get_all())

[[NeuronText(score=0.6828351020812988, text='Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'), NeuronText(score=0.6577122807502747, text='Once upon a time, there was a little girl named Lily. Lily liked to pretend she was a popular princess. She lived in a big castle with her best friends, a cat and a dog.\n\nOne day, while playing in 